In [ ]:
import numpy as np
import torch
import art.attacks.evasion as toolbox
from art.estimators.classification import PyTorchClassifier
from kd_export import LightNN
from mvtec import test_dataset

In [ ]:
print("Check current device: ")
# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): # Should return True 
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # Should show your GPU name
else:
    print("Using CPU")

In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrained_model = "student_model.pth"
pretrained_model_KD = "student_model_KD.pth"
use_cuda=True
# Set random seed for reproducibility
torch.manual_seed(42)

In [ ]:
# Initialize the network
torch.manual_seed(42)
model = LightNN(num_classes=15).to(device)
torch.manual_seed(42)
model_KD = LightNN(num_classes=15).to(device)


# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model, map_location=device, weights_only=True))
model_KD.load_state_dict(torch.load(pretrained_model_KD, map_location=device, weights_only=True))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()
model_KD.eval()

In [ ]:
# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Wrap the model with PyTorchClassifier
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),  # Input values range, e.g., for normalized images
    loss=loss_fn,
    optimizer=optimizer,
    input_shape=(3, 224, 224),  # Adjust to your input shape
    nb_classes=10              # Number of output classes
)

In [ ]:
x_test = np.array([data[0].numpy() for data in test_dataset])
y_test = np.array([data[1] for data in test_dataset])

In [ ]:
# Create FGSM attack
attack = toolbox.FastGradientMethod(estimator=classifier, eps=0.1)

# Generate adversarial examples
x_test_adv_fgsm = attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_fgsm)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on FSGM attack: {accuracy * 100:.2f}%")

In [ ]:
# Create PGD attack
pgd_attack = toolbox.ProjectedGradientDescent(estimator=classifier, eps=0.1, eps_step=0.01, max_iter=40)

# Generate adversarial examples
x_test_adv_pgd = pgd_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_pgd)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on PGD attack: {accuracy * 100:.2f}%")

In [ ]:
# Create C&W attack
cw2_attack = toolbox.CarliniL2Method(estimator=classifier, confidence=0.0, max_iter=40, learning_rate=0.01)

# Generate adversarial examples
x_test_adv_cw2 = cw2_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_cw2)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on CW2 attack: {accuracy * 100:.2f}%")

In [ ]:
# Create Universal Perturbations attack
up_attack = toolbox.UniversalPerturbation(estimator=classifier,attacker="deepfool",
                                          attacker_params={"eps": 0.1},
                                          delta=0.2,
                                          max_iter=10,
                                          eps=0.1,
                                          batch_size=32,
                                          verbose=True)

# Generate adversarial examples
x_test_adv_up = up_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_up)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on UP attack: {accuracy * 100:.2f}%")

In [ ]:
# Create Salt-and-Pepper Noise attack
up_attack = toolbox

# Generate adversarial examples
x_test_adv_up = up_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_up)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on UP attack: {accuracy * 100:.2f}%")